In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, confusion_matrix, mean_squared_error, mean_absolute_error
from sksurv.metrics import concordance_index_censored, brier_score, cumulative_dynamic_auc
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import glob
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import RandomOverSampler
import shap
from sksurv.nonparametric import kaplan_meier_estimator
from sklearn.utils.class_weight import compute_sample_weight

seed = 1

# Define custom scoring function for the concordance index
def concordance_scorer(estimator, X, y):
    # Extract event indicator and time from the structured array
    event_indicator, event_time = y["Cancer_Status"], y["TimeYears_CT_blood"]
    
    # Predict risk scores
    risk_scores = estimator.predict(X)
    
    # Calculate concordance index
    concordance = concordance_index_censored(event_indicator, event_time, risk_scores)[0]
    return concordance

def brier_score_at_time(estimator, X, y, time_point=0):
    """Custom Brier Score metric at a specific time point for use in GridSearchCV."""
    surv_funcs = estimator.predict_survival_function(X)  # Get survival functions for the predictions
    test_estimates = np.array([fn(time_point) for fn in surv_funcs])  # Survival prob. at 1 year
    brier_score_1yr = brier_score(y_train, y, test_estimates, [time_point])[1][0]  # Brier Score at 1 year
    return -brier_score_1yr  # Negative for maximizing in GridSearchCV

np.set_printoptions(precision=3)
n_folds=3
path_to_save_results = '/home/ubuntu/tenerife/data/LungAmbition/Proteinas/ResultsBaselines_BM/RSF'
path_to_save_plots = os.path.join(path_to_save_results, 'plots')
path_to_fold_division_folder = f'/home/ubuntu/tenerife/data/ZZ_githubRepos/LungAmbition/Data_stratified_split/folds-def_{n_folds}folds'
keep_false_positives_as_separate_test = True
# create path_to_save_plots if it does not exist
if not os.path.exists(path_to_save_plots):
    os.makedirs(path_to_save_plots)

name_file= "ROS_RSF_proteins_6nov2024_correctedTo5YearsCensorship"
if keep_false_positives_as_separate_test:
    name_file= name_file + "_keep_false_positives_as_separate_test"
sys.stdout=open(os.path.join(path_to_save_results, "run_out_"+name_file+"_5fold.txt"),'w')
# save prints in a txt file
original_stdout = sys.stdout

df_merged = pd.read_csv('/home/ubuntu/tenerife/data/LungAmbition/Excels_merged/LungAmbitionMergedAll7nov2024.csv')
# ignore columns 'ID_imagingData', 'Group', 'Cancer_Status', 'TimeYears_CT_blood',
# 'Diff_Diag_Blood_TimeYears_CT_blood', 'LastFollow_upTimeYears_CT_blood', 'Age', 'Sex', 
# 'Body_mass_index','Smoking_status', 'Years_smoked', 'Smoking_pack_years', 
# 'Family_history_lung_cancer','Personal_history_cancer', 'Stage_category','NRRD_File', 'SEG_Files'
# in df_merged
# for correct handling of TimeYears_CT_blood, if TimeYears_CT_blood is over 5 years, change it to 5 years
# in df_merged column TimeYears_CT_blood change to 5 if its over 5 years for patients that are not in Lung_Cancer group
df_merged.loc[(df_merged['TimeYears_CT_blood'] == 5) & (df_merged['Group'] != 'Lung_Cancer'), 'TimeYears_CT_blood'] = 6
y_target = df_merged[['ID_proteinData', 'Cancer_Status', 'TimeYears_CT_blood']]
columns_to_drop=['ID_imagingData', 'Group', 'Cancer_Status', 
                'TimeYears_blood', 'TimeMonths_blood', 'TimeYears_CT_blood', 'TimeMonths_CT_blood',
                'Diff_Diag_Blood_TimeYears', 'LastFollow_upTimeYears', 
                'Age', 'Sex', 'Body_mass_index','Smoking_status', 'Years_smoked', 'Smoking_pack_years',
                'Family_history_lung_cancer','Personal_history_cancer', 'Stage_category',
                'NRRD_File', 'SEG_Files']
# for correct handling of TimeYears_CT_blood, if TimeYears_CT_blood is over 5 years, change it to 5 years
# in df_merged column TimeYears_CT_blood change to 5 if its over 5 years for patients that are not in Lung_Cancer group

X = df_merged.drop(columns=columns_to_drop)
if keep_false_positives_as_separate_test:
    # create list to store wrong predicted false positives
    list_ID_wrong_predicted_false_positives = []
    # save false_positive_metrics in df

# save best metrics for each fold
fold_metrics_df = pd.DataFrame(columns=['Fold', 'Concordance_Index', 'Integrated_Brier_Score',
                                        'AUC_at_0_Years', 'Accuracy_at_0_Years', 'Balanced_Accuracy_at_0_Years', 'Precision_at_0_Years', 'Recall_at_0_Years', 'F1_Score_at_0_Years', 'Specificity_at_0_Years', 'NPV_at_0_Years',
                                        'AUC_at_1_Years', 'Accuracy_at_1_Years', 'Balanced_Accuracy_at_1_Years', 'Precision_at_1_Years', 'Recall_at_1_Years', 'F1_Score_at_1_Years', 'Specificity_at_1_Years', 'NPV_at_1_Years',
                                        'AUC_at_2_Years', 'Accuracy_at_2_Years', 'Balanced_Accuracy_at_2_Years', 'Precision_at_2_Years', 'Recall_at_2_Years', 'F1_Score_at_2_Years', 'Specificity_at_2_Years', 'NPV_at_2_Years',
                                        'AUC_at_3_Years', 'Accuracy_at_3_Years', 'Balanced_Accuracy_at_3_Years', 'Precision_at_3_Years', 'Recall_at_3_Years', 'F1_Score_at_3_Years', 'Specificity_at_3_Years', 'NPV_at_3_Years',
                                        'AUC_at_4_Years', 'Accuracy_at_4_Years', 'Balanced_Accuracy_at_4_Years', 'Precision_at_4_Years', 'Recall_at_4_Years', 'F1_Score_at_4_Years', 'Specificity_at_4_Years', 'NPV_at_4_Years',
                                        'AUC_at_5_Years', 'Accuracy_at_5_Years', 'Balanced_Accuracy_at_5_Years', 'Precision_at_5_Years', 'Recall_at_5_Years', 'F1_Score_at_5_Years', 'Specificity_at_5_Years', 'NPV_at_5_Years'])
# save top features for each fold
top_features_df_list = []

# iterate over each file in path_to_fold_division_folder
# Use glob to find all CSV files in the folder
for file_path in sorted(glob.glob(os.path.join(path_to_fold_division_folder, 'id2splitfold_*.csv'))):
    print("Reading file:", file_path)
    # Extract the fold number from the filename
    fold = int(os.path.basename(file_path).split('_')[-1].split('.')[0])
    # Load the CSV file into a DataFrame
    df_fold = pd.read_csv(file_path)
    # rename GroupUpdated column into Group in df_fold
    df_fold.rename(columns={'GroupUpdated': 'Group'}, inplace=True)
    # drop columns where Group == Control
    df_fold = df_fold[df_fold['Group'] != 'Control'] ### uncomment this for BC vs LC
    # rename in Group column IndeterminatePreLungCancer to Lung_Cancer in df_fold
    df_fold.loc[df_fold['Group'] == 'IndeterminatePreLungCancer', 'Group'] = 'Lung_Cancer'
    print(df_fold[df_fold['Group']=='Lung_Cancer'].TimeYears_CT_blood.value_counts())
    # next line is not necfcessary, but we keep here for completion
    df_fold.loc[(df_fold['TimeYears_CT_blood'] == 5) & (df_fold['Group'] != 'Lung_Cancer'), 'TimeYears_CT_blood'] = 6
    print("=" * 80)
    print(f"Fold {fold + 1}:")
    # based on df_fold.split (train, test, val, test_false_positive) asign in df_merged the corresponding fold
    # Get the ID_proteinData values for each split in df_fold
    train_ids = df_fold[df_fold['split'] == 'train']['ID_proteinData'].tolist()
    val_ids = df_fold[df_fold['split'] == 'val']['ID_proteinData'].tolist()
    test_ids = df_fold[df_fold['split'] == 'test']['ID_proteinData'].tolist()
    if keep_false_positives_as_separate_test:
        false_positive_ids = df_fold[df_fold['split'] == 'test_false_positive']['ID_proteinData'].tolist()
        # Get the corresponding rows from df_merged
        X_test_false_positives = df_merged[df_merged['ID_proteinData'].isin(false_positive_ids)]
        y_test_false_positives = df_merged[df_merged['ID_proteinData'].isin(false_positive_ids)][['Cancer_Status', 'TimeYears_CT_blood']]
        ID_false_positives = X_test_false_positives['ID_proteinData'].values
        X_test_false_positives = X_test_false_positives.drop(columns=columns_to_drop)
        X_test_false_positives = X_test_false_positives.drop(columns=['ID_proteinData'])
        print("False positives data:", X_test_false_positives.shape, y_test_false_positives.shape, 
              "Test false positives:", len(y_test_false_positives[y_test_false_positives.Cancer_Status == 0]))
    else:
        # include false positives in test
        test_ids = test_ids + df_fold[df_fold['split'] == 'test_false_positive']['ID_proteinData'].tolist()

    # Filter X and y_target based on these lists of IDs
    X_train = X[X['ID_proteinData'].isin(train_ids)].drop(columns=['ID_proteinData'])
    X_val = X[X['ID_proteinData'].isin(val_ids)].drop(columns=['ID_proteinData'])
    X_test = X[X['ID_proteinData'].isin(test_ids)].drop(columns=['ID_proteinData'])
    # ver que hacer con false_positives, eliminar ID_proteinData
    y_train = y_target[y_target['ID_proteinData'].isin(train_ids)].drop(columns=['ID_proteinData'])
    y_val = y_target[y_target['ID_proteinData'].isin(val_ids)].drop(columns=['ID_proteinData'])
    y_test = y_target[y_target['ID_proteinData'].isin(test_ids)].drop(columns=['ID_proteinData'])

    print("Training data:", X_train.shape, y_train.shape, 
          "Benigns train:", len(y_train[y_train.Cancer_Status == 0]), "Lung cancer patients train:", len(y_train[y_train.Cancer_Status == 1]))
    print("Validation data:", X_val.shape, y_val.shape, 
          "Benigns val:", len(y_val[y_val.Cancer_Status == 0]), "Lung cancer patients val:", len(y_val[y_val.Cancer_Status == 1]))
    print("Test data:", X_test.shape, y_test.shape, 
          "Benigns test:", len(y_test[y_test.Cancer_Status == 0]), "Lung cancer patients test:", len(y_test[y_test.Cancer_Status == 1]))
    print("Number of samples per class in y_train")
    print(y_train.Cancer_Status.value_counts())
    print("Number of samples per class in y_test")
    print(y_test.Cancer_Status.value_counts())
    print("Number of samples per class in y_val")
    print(y_val.Cancer_Status.value_counts())
    # join X_val in X_train and y_val in y_train ### OJO esto revisar!!
    X_train = pd.concat([X_train, X_val], axis=0)
    y_train = pd.concat((y_train, y_val), axis=0)
    # print max TimeYears_CT_blood in y_train
    print("Max TimeYears_CT_blood in y_train:", y_train['TimeYears_CT_blood'].max())
    print("Max TimeYears_CT_blood in y_test:", y_test['TimeYears_CT_blood'].max())
    # print training data after joining with validation data
    print("Training data after joining with validation data:", X_train.shape, y_train.shape, "Benign patients train:", len(y_train[y_train['Cancer_Status'] == False]), "Lung cancer patients train:", len(y_train[y_train['Cancer_Status'] == True]))
    print("Lung cancer patients (TimeYears_CT_blood = 0, before duplicating):", len(y_train[(y_train['Cancer_Status'] == 1) & (y_train['TimeYears_CT_blood'] == 0)]))
    print("Lung cancer patients (TimeYears_CT_blood > 0, before duplicating):", len(y_train[(y_train['Cancer_Status'] == 1) & (y_train['TimeYears_CT_blood'] > 0)]))
    print("Benign participants:", len(y_train[y_train['Cancer_Status'] == 0]))
    # combine X_train and y_train to apply oversampling and oversample only the lung cancer class
    X_y_train_combined = pd.concat([X_train, y_train], axis=1)
    # drop column Cancer_Status in X_y_train_combined
    y_train_CancerStatus = X_y_train_combined['Cancer_Status'].values
    # duplicate all LC patients to improve learning
    # in X_y_train_combined_TimeYears_CT_blood repeat twice rows where LungCancer == 1 and TimeYears_CT_blood == 0
    filtered_rows = X_y_train_combined[
        (X_y_train_combined['Cancer_Status'] == 1)]

    # Repeat these rows twice
    resampled_rows = pd.concat([filtered_rows], ignore_index=True)

    # Append to the original DataFrame
    X_y_train_resampled_TimeYears_CT_blood = pd.concat(
        [X_y_train_combined, resampled_rows], ignore_index=True
    )
    X_train_resampled = X_y_train_resampled_TimeYears_CT_blood.drop(columns=['TimeYears_CT_blood', 'Cancer_Status'])
    y_train_resampled = X_y_train_resampled_TimeYears_CT_blood[['TimeYears_CT_blood', 'Cancer_Status']]
    print("Training data after duplicating LC patients:", X_train_resampled.shape, y_train_resampled.shape)
    # rename X_train_resampled to X_train and y_train_resampled to y_train
    X_train = X_train_resampled
    y_train = y_train_resampled
    print("Non-lung cancer patients (All years):", len(y_train[y_train == 0]))
    print("Lung cancer patients (TimeYears_CT_blood = 0):", len(y_train[(y_train['Cancer_Status'] == 1) & (y_train['TimeYears_CT_blood'] == 0)]))
    print("Lung cancer patients (TimeYears_CT_blood > 0):", len(y_train[(y_train['Cancer_Status'] == 1) & (y_train['TimeYears_CT_blood'] > 0)]))
    print("Benign participants:", len(y_train[y_train['Cancer_Status'] == 0]))

    # # print("y_train_CancerStatus after conversion:", y_train_CancerStatus)
    # X_y_train_combined_TimeYears_CT_blood = X_y_train_combined.drop(columns=['Cancer_Status'])

    # # Define RandomOverSampler to target the minority class in `Cancer_Status`
    # ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)

    # # Apply oversampling on the combined DataFrame
    # X_y_train_resampled_TimeYears_CT_blood, y_train_resampled_CancerStatus = ros.fit_resample(X_y_train_combined_TimeYears_CT_blood, y_train_CancerStatus)
    # # print("y_train_resampledCancerStatus after conversion:", y_train_resampled_CancerStatus)
    # # Split the resampled data into X and y parts
    # y_train_resampled = pd.DataFrame(y_train_resampled_CancerStatus, columns=['Cancer_Status'])
    # # include in y_train_resampled the TimeYears_CT_blood
    # y_train_resampled['TimeYears_CT_blood'] = X_y_train_resampled_TimeYears_CT_blood['TimeYears_CT_blood']
    # X_train_resampled = X_y_train_resampled_TimeYears_CT_blood.drop(columns=['TimeYears_CT_blood'])  # All columns except 'TimeYears_CT_blood'

    # print("Training data after oversampling:", X_train_resampled.shape, y_train_resampled.shape)
    # print("Non lung cancer patients train:", len(y_train_resampled[y_train_resampled['Cancer_Status'] == False]))
    # print("Lung cancer patients train:", len(y_train_resampled[y_train_resampled['Cancer_Status'] == True]))

    # # rename X_train_resampled to X_train and y_train_resampled to y_train
    # X_train = X_train_resampled
    # y_train = y_train_resampled
    # oversample lung cancer at year 0 class
    # Identify patients where `TimeYears_CT_blood == 0`
        
    event = y_train["Cancer_Status"]  # Structured array access
    time = y_train["TimeYears_CT_blood"]  # Structured array access
    
    # convert y to structured array with the first field being a binary class event indicator and the second field the time of the event/censoring
    y_train = np.array([(status, time) for status, time in y_train.values], dtype=[('Cancer_Status', 'bool'), ('TimeYears_CT_blood', 'float')])
    y_test = np.array([(status, time) for status, time in y_test.values], dtype=[('Cancer_Status', 'bool'), ('TimeYears_CT_blood', 'float')])
    
    # scale the data using StandardScaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    # print shapes of X_train and X_test
    print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
    print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)
    # train RSF, first find the best hyperparameters through a grid search cv,
    # then train the model with the best hyperparameters
    # parameters to search
    param_grid = {
        "n_estimators": [100, 200, 300, 400],
        "max_depth": [2, 3, 4, 5],
        "min_samples_split": [5, 6, 7],
        "min_samples_leaf": [4, 5],
        "max_features": ['sqrt', 150, 200, 250, 'log2'],
    }
    # random forest classifier
    rsf = RandomSurvivalForest(random_state=seed)
    # grid search cv
    # Set up GridSearchCV with validation
    grid_search = GridSearchCV(
        estimator=rsf,
        param_grid=param_grid,
        cv=KFold(n_splits=3, shuffle=True, random_state=seed),
        n_jobs=-1,
        verbose=0,
        scoring={'concordance': concordance_scorer, 'brier_0yr': brier_score_at_time},
        refit='brier_0yr'  # You can refit on the metric that prioritizes your objective
    )
    # sample_weights = compute_sample_weight(class_weight='balanced', y=(event.astype(bool)) & (time == 0))
    # Fit the grid search on training data
    grid_search.fit(X_train, y_train)#, sample_weight=sample_weights)

    # Print the best hyperparameters
    print("Best hyperparameters:")
    print(grid_search.best_params_)
    best_score = grid_search.best_score_
    print(f"Best Brier Score at 0 year: {-best_score}")
    # print("Concordance Index scores for each parameter set:\n", grid_search.cv_results_['mean_test_concordance'])
    # Use the best estimator to train the model
    best_rsf = grid_search.best_estimator_
    best_rsf.fit(X_train, y_train)#, sample_weight=sample_weights)
    
    # Predict risk scores for train and validation sets
    train_risk_scores = best_rsf.predict(X_train)
    # Concordance Index
    train_concordance = concordance_index_censored(y_train["Cancer_Status"], y_train["TimeYears_CT_blood"], train_risk_scores)[0]
    # Print metrics for train and validation sets
    print("Metrics in train:")
    print(f"Concordance Index: {train_concordance}")
    print("=" * 80)

    # predict in test
    y_pred = best_rsf.predict(X_test)

    # Calculate the concordance index
    concordance = concordance_index_censored(y_test["Cancer_Status"], y_test["TimeYears_CT_blood"], y_pred)[0]

    # Predict the survival functions for each sample in the training set
    surv_funcs = best_rsf.predict_survival_function(X_test)
    # output is a is a list of survival functions, one for each sample in X_test. 
    # Each survival function provides the probability of survival.
    # These probabilities range from 1 (100% survival) at the start of the study period 
    # down to lower values as time progresses, representing the likelihood that the event 
    # has not yet occurred by a given time.

    # Compute censoring survival function
    time_censoring, survival_censoring = kaplan_meier_estimator(event= y_train["Cancer_Status"], time_exit=y_train["TimeYears_CT_blood"])

    # Plot censoring survival function
    plt.step(time_censoring, survival_censoring, where="post", label="Censoring Survival Function")
    plt.xlabel("Time")
    plt.ylabel("Survival Probability")
    plt.title("Kaplan-Meier Censoring Survival Curve")
    plt.savefig(os.path.join(path_to_save_plots, f"KM_Censoring_Surv_fold_{fold+1}.png"))
    plt.close()

    # print("surv_funcs:", surv_funcs)
    # time_points = [0, 1, 2, 3, 4, 5]  # Specify the time points at which to calculate the Brier score
    time_points = [float(t) for t in [0, 1, 2, 3, 4, 5]]
    print(f"Min time: {y_test['TimeYears_CT_blood'].min()}, Max time: {y_test['TimeYears_CT_blood'].max()}")
    print("y_pred", y_pred)
    # Compute time-dependent AUC
    auc_scores = cumulative_dynamic_auc(y_train, y_test, y_pred, time_points)

/home/ubuntu/tenerife/mambaforge/envs/lungAmbpy3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: censoring survival function is zero at one or more time points

In [2]:
y_train

array([( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.),
       ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.),
       ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.),
       ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.),
       ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.),
       ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.),
       ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.),
       ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.),
       ( True, 1.), ( True, 1.), ( True, 1.), ( True, 1.), ( True, 1.),
       ( True, 1.), (False, 1.), (False, 1.), ( True, 1.), (False, 1.),
       (False, 1.), (False, 1.), (False, 1.), (False, 1.), (False, 1.),
       ( True, 1.), (False, 1.), ( True, 1.), (False, 1.), ( True, 1.),
       ( True, 1.), (False, 1.), ( True, 1.), (False, 1.), (False, 1.),
       (False, 1.), ( True, 1.), ( True, 1.), ( True, 1.), ( Tru

In [3]:
y_test

array([(False, 6.), (False, 6.), (False, 6.), (False, 6.), (False, 6.),
       (False, 6.), (False, 6.), (False, 6.), (False, 6.), (False, 6.),
       (False, 6.), (False, 6.), (False, 6.), (False, 6.), (False, 6.),
       (False, 6.), (False, 6.), (False, 6.), (False, 6.), (False, 6.),
       (False, 6.), (False, 6.), (False, 6.), ( True, 1.), ( True, 1.),
       ( True, 2.), ( True, 0.), ( True, 0.), ( True, 5.), ( True, 3.),
       ( True, 4.), ( True, 4.), ( True, 4.), ( True, 0.), ( True, 3.),
       ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.), ( True, 0.),
       ( True, 0.), ( True, 0.), ( True, 0.), ( True, 2.), ( True, 1.)],
      dtype=[('Cancer_Status', '?'), ('TimeYears_CT_blood', '<f8')])